In [1]:
import os
import sys
import re
import time
from pathlib import Path
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import warnings
import logging
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

sys.path.append('../python')
warnings.filterwarnings('default')
logging.getLogger("pdfminer").setLevel(logging.ERROR)

import api

rng = np.random.default_rng(12898)


/Users/ekung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ekung/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PROMPT = """
==== LIST OF AGENDA ITEMS ====

{}

==== DOCUMENT ====

{}

==== PROMPT ====

I just gave you a list of agenda items from a LA City Planning Commission meeting, followed by a document submitted to that meeting. 

Return a response in the following format:


==== YOUR RESPONSE FORMAT ====

TYPE OF DOCUMENT:
<What type of document is it? Your only options are: LETTER OR PETITION, TECHNICAL MODIFICATION OR PROCEDURAL MATTER, SCIENTIFIC OR TECHNICAL REPORT, CV OR BIOGRAPHY, CORRUPTED/ILLEGIBLE/BLANK, TITLE OR SECTION HEADING, OTHER.>

TYPE OF AUTHOR:
<What type of entity wrote the document? Your only options are: INDIVIDUAL, ADVOCACY GROUP, CONSULTANT, LAWYER, DEVELOPER, PUBLIC OFFICIAL, OTHER.>

SUMMARY OF DOCUMENT:
<Summarize the contents of the document.>

REFERENCED AGENDA ITEMS:
<List the agenda items, as a comma delimited list of item numbers, that the submitted document references or is relevant to. If none, say NONE.>

SUPPORT OR OPPOSE:
<Does the submitted document support or oppose the referenced agenda items? Your only options are: DEFINITELY SUPPORT, SOMEWHAT SUPPORT, DEFINITELY OPPOSE, SOMEWHAT OPPOSE, NEUTRAL, NOT RELEVANT.>
"""


In [3]:
meta_df = pd.read_csv("../../intermediate_data/cpc/meetings_metadata.csv")

In [4]:
t0 = time.time()
for i, irow in meta_df.iterrows():
    date = irow['date']
    year = irow['year']
    print(date)

    agenda_file = f"../../intermediate_data/cpc/{year}/{date}/agenda-items.pkl"
    docs_file = f"../../intermediate_data/cpc/{year}/{date}/supplemental-docs.pkl"
    output_file = f"../../intermediate_data/cpc/{year}/{date}/supplemental-docs-summaries.pkl"

    if (not os.path.exists(agenda_file)) or (not os.path.exists(docs_file)):
        continue

    out_df = []

    agenda_df = pd.read_pickle(agenda_file)
    agenda = ""
    for j, jrow in agenda_df.iterrows():
        item_no = jrow['item_no']
        item_title = jrow['title']
        is_casenum = jrow['is_casenum']
        if is_casenum:
            text = jrow['content']
            agenda += f"ITEM NO. {item_no}\n{item_title}\n\n"
            agenda += text
            agenda += "\n\n--------\n\n"
        
    docs_df = pd.read_pickle(docs_file)
    docs_df['skipped'] = docs_df['content'].str.startswith('SKIPPED')
    for j, jrow in docs_df.loc[~docs_df['skipped']].iterrows():
        doc_id = jrow['doc_id']
        start_page = jrow['start_page']
        end_page = jrow['end_page']
        content = jrow['content']
        prompt = PROMPT.format(agenda, content)
        response, score = api.get_response(prompt)
        out_df.append({
            'year': year,
            'date': date,
            'doc_id': doc_id,
            'start_page': start_page,
            'end_page': end_page,
            'prompt': prompt,
            'response': response,
            'score': score
        })
        print(f"{end_page}... ", end='')

    out_df = pd.DataFrame.from_dict(out_df)
    out_df.to_pickle(output_file)
    print('')
t1 = time.time()
print(f"Elapsed time: {(t1-t0)/60} minutes.")


2018-05-10
1... 2... 8... 9... 10... 174... 176... 178... 179... 188... 189... 193... 207... 208... 209... 210... 211... 212... 213... 214... 215... 216... 218... 219... 
2018-05-23
1... 2... 3... 4... 6... 7... 20... 30... 31... 
2018-06-14
1... 2... 3... 5... 7... 9... 17... 21... 24... 31... 43... 47... 51... 61... 62... 66... 71... 77... 78... 80... 81... 
2018-07-12
1... 2... 3... 4... 9... 12... 13... 16... 21... 23... 25... 26... 34... 38... 43... 50... 59... 64... 66... 67... 68... 69... 71... 74... 75... 
2018-07-26
1... 2... 4... 5... 6... 7... 8... 9... 10... 11... 12... 14... 15... 16... 17... 18... 19... 20... 21... 22... 23... 24... 25... 26... 27... 30... 31... 32... 35... 36... 37... 39... 40... 41... 42... 43... 44... 45... 46... 47... 48... 49... 
2018-08-09
1... 2... 6... 10... 12... 14... 23... 29... 30... 31... 32... 33... 34... 57... 66... 67... 89... 90... 133... 134... 135... 136... 140... 141... 143... 144... 148... 149... 150... 152... 153... 154... 155... 
20

In [6]:
i=3
print(out_df.loc[i, 'prompt'])
print('')
print('')
print(out_df.loc[i, 'response'])


==== LIST OF AGENDA ITEMS ====

ITEM NO. 5a
CPC-2023-7708-DB-CDO-HCA

     5a. CPC-2023-7708-DB-CDO-HCA                       Council District: 13 – Soto-Martinez
         CEQA: ENV-2023-7709-CE                               Last Day to Act: 10-10-24
         Plan Area: Northeast Los Angeles                                           
                                                                                    
         PUBLIC HEARING – Completed September 30, 2024                              
                                                                                    
         PROJECT SITE:   4002 – 4006 North Verdugo Road; 3067 North Delevan Drive   
                                                                                    
         PROPOSED PROJECT:                                                          
         Construction, use, and maintenance of a new, three-story over one basement-story mixed-use building
         totaling 18,799 square feet pursuan

In [ ]:
print(out_df.loc[22, 'response'])